* [University Digital Conservancy - UDC](https://conservancy.umn.edu/) 
* [Data Repository of UM - DRUM](https://conservancy.umn.edu/handle/11299/166578) 

Both of the above use the Dspace platform. 
See [the DSpace 7 API documentation here](https://wiki.lyrasis.org/display/DSDOC7x/REST+API).
#### Live API, when launched publicly
- Direct access to the UMN Digital Conservancy's instance of the REST API will be available at: [https://conservancy.umn.edu/server/api](https://conservancy.umn.edu/server/api)
- There is a helper tool called the [HAL browser](https://conservancy.umn.edu/server/) that shows what calls are available in a more human readable version.
#### Development Test Instance
- **Test Instance** of the REST API (must be connected to VPN Split Tunnel) is available at: [https://udc-prd.lib.umn.edu/server/api](https://udc-prd.lib.umn.edu/server/api)
- **Test Instance** of helper tool called the [HAL browser](https://udc-prd.lib.umn.edu/server) that shows what calls are available in a more human readable version.

Some sample exploratory code below.


In [11]:
# about the requests library - https://requests.readthedocs.io/en/latest/user/quickstart/
import requests
# about the json library - https://docs.python.org/3/library/json.html
import json
import urllib.request

In [2]:
# old version: response = requests.get('https://conservancy.umn.edu/rest/test')
# old version: response.content

response = requests.get('https://udc-prd.lib.umn.edu/server/api')
response_test = response.status_code
print(response_test)

if response_test == 200:
    print('API connection is successful')
else:
    print('Error connecting to API, try a different url')


200
API connection is successful


In [17]:
# base_url = 'https://conservancy.umn.edu'
base_url = 'https://udc-prd.lib.umn.edu' #this is the link to use for testing

# old REST endpoint from DSpace 5.11  rest_url = 'https://conservancy.umn.edu/rest'
# when DSpace7 goes live the link will be 'https://conservancy.umn.edu/server/api'
rest_url = 'https://udc-prd.lib.umn.edu/server/api/core'

We need to get some information about the item we are going to use in this exercise.

Links with handles will need one more step to be able to interact with the API.
We need to find out if the persistent link is associated with an 'item', 'collection', or 'community'.
After entering the link we are going to use, we need to convert it to a url that the API can use.

In [9]:
example_url = input()
print(example_url)

https://udc-prd.lib.umn.edu/handle/11299/180876


In [13]:
resolved_url = requests.get(example_url) #when Dspace7 is live the handle url should resolve to the new uuid-based url
r_new_url = resolved_url.url
print(r_new_url)

https://udc-prd.lib.umn.edu/communities/bf09d885-4a3c-48dc-83a5-f29f9d8e78d8


The url link structure now includes a unique id at the end or 'uuid'. 
We need to incorporate the 'item', 'collection', or 'community' and the 'uuid' into the API call.

In [21]:
end_url_split = r_new_url.split ("/") [-2:]
url_type = end_url_split[0]
print('This Handle URL is a part of DSpace: ' + url_type)
url_uuid = end_url_split[1]
print('UUID is: ' + url_uuid)

This Handle URL is a part of DSpace: communities
UUID is: bf09d885-4a3c-48dc-83a5-f29f9d8e78d8


In [26]:
# Bee Lab community: https://udc-prd.lib.umn.edu/handle/11299/180876
# Handles are still used in DSpace7 but the Handle link will resolve to this URL: 
# https://udc-prd.lib.umn.edu/communities/bf09d885-4a3c-48dc-83a5-f29f9d8e78d8

rest_api_url = rest_url + '/' + url_type + '/' + url_uuid
print(rest_api_url)
bee_lab = requests.get(rest_api_url)
bee_lab_json = bee_lab.content
bee_lab_json


https://udc-prd.lib.umn.edu/server/api/core/communities/bf09d885-4a3c-48dc-83a5-f29f9d8e78d8


b'{\n  "id" : "bf09d885-4a3c-48dc-83a5-f29f9d8e78d8",\n  "uuid" : "bf09d885-4a3c-48dc-83a5-f29f9d8e78d8",\n  "name" : "Bee Lab",\n  "handle" : "11299/180876",\n  "metadata" : {\n    "dc.identifier.uri" : [ {\n      "value" : "http://localhost:4000/handle/11299/180876",\n      "language" : null,\n      "authority" : null,\n      "confidence" : -1,\n      "place" : 0\n    } ],\n    "dc.title" : [ {\n      "value" : "Bee Lab",\n      "language" : null,\n      "authority" : null,\n      "confidence" : -1,\n      "place" : 0\n    } ]\n  },\n  "archivedItemsCount" : 38,\n  "type" : "community",\n  "_links" : {\n    "collections" : {\n      "href" : "https://udc-prd.lib.umn.edu/server/api/core/communities/bf09d885-4a3c-48dc-83a5-f29f9d8e78d8/collections"\n    },\n    "logo" : {\n      "href" : "https://udc-prd.lib.umn.edu/server/api/core/communities/bf09d885-4a3c-48dc-83a5-f29f9d8e78d8/logo"\n    },\n    "subcommunities" : {\n      "href" : "https://udc-prd.lib.umn.edu/server/api/core/communi

We now need to navigate the Communities JSON response to find its collections

In [35]:
# snag the "collection" id and link for a community from the dictionary returned
# use json.loads(variable.content.decode('utf-8')) to convert a byte object to json and then call an item ['_links']

beeLabLinks = json.loads(bee_lab_json.decode('utf-8'))['_links']
print(beeLabLinks)
# navigate the JSON response of the community to find its collections
beeLabCollections = beeLabLinks['collections']
collection_link = beeLabCollections['href']
collection_link

{'collections': {'href': 'https://udc-prd.lib.umn.edu/server/api/core/communities/bf09d885-4a3c-48dc-83a5-f29f9d8e78d8/collections'}, 'logo': {'href': 'https://udc-prd.lib.umn.edu/server/api/core/communities/bf09d885-4a3c-48dc-83a5-f29f9d8e78d8/logo'}, 'subcommunities': {'href': 'https://udc-prd.lib.umn.edu/server/api/core/communities/bf09d885-4a3c-48dc-83a5-f29f9d8e78d8/subcommunities'}, 'parentCommunity': {'href': 'https://udc-prd.lib.umn.edu/server/api/core/communities/bf09d885-4a3c-48dc-83a5-f29f9d8e78d8/parentCommunity'}, 'adminGroup': {'href': 'https://udc-prd.lib.umn.edu/server/api/core/communities/bf09d885-4a3c-48dc-83a5-f29f9d8e78d8/adminGroup'}, 'self': {'href': 'https://udc-prd.lib.umn.edu/server/api/core/communities/bf09d885-4a3c-48dc-83a5-f29f9d8e78d8'}}


'https://udc-prd.lib.umn.edu/server/api/core/communities/bf09d885-4a3c-48dc-83a5-f29f9d8e78d8/collections'

In [22]:
# snag the "collection" id and link for a community from the dictionary returned
# use json.loads(variable.content.decode('utf-8')) to convert a byte object to json and then call an item ['link']
link = json.loads(bee_lab.content.decode('utf-8'))['link']
bee_lab_new = requests.get(base_url + str(link) + '/collections')
#print(bee_lab_new.content)
collection_link = json.loads(bee_lab_new.content.decode('utf-8'))[0]['link']
#collection_link

KeyError: 'link'

In [78]:
# use a collection link (/rest/collections/1268/items) to get a list of all items
bee_lab_items = requests.get(base_url + collection_link + '/items')
bee_lab_items.content

b'[{"id":76619,"name":"A comparison of honey bee-collected pollen from working agricultural lands using light microscopy and ITS metabarcoding","handle":"11299/197612","type":"item","link":"/rest/items/76619","expand":["metadata","parentCollection","parentCollectionList","parentCommunityList","bitstreams","all"],"lastModified":"2018-05-31 11:07:52.917","parentCollection":null,"parentCollectionList":null,"parentCommunityList":null,"bitstreams":null,"archived":"true","withdrawn":"false"},{"id":76618,"name":"Effects of neonicotinoid imidacloprid exposure on bumble bee (Hymenoptera: Apidae) queen survival and nest initiation","handle":"11299/197611","type":"item","link":"/rest/items/76618","expand":["metadata","parentCollection","parentCollectionList","parentCommunityList","bitstreams","all"],"lastModified":"2018-05-31 11:07:29.007","parentCollection":null,"parentCollectionList":null,"parentCommunityList":null,"bitstreams":null,"archived":"true","withdrawn":"false"},{"id":76617,"name":"Sta

In [86]:
# call a single item and convert to json
bee_lab_item = requests.get(rest_url + '/handle/11299/197612')
item_json = json.loads(bee_lab_item.content.decode('utf-8'))
item_json

{'id': 76619,
 'name': 'A comparison of honey bee-collected pollen from working agricultural lands using light microscopy and ITS metabarcoding',
 'handle': '11299/197612',
 'type': 'item',
 'link': '/rest/items/76619',
 'expand': ['metadata',
  'parentCollection',
  'parentCollectionList',
  'parentCommunityList',
  'bitstreams',
  'all'],
 'lastModified': '2018-05-31 11:07:52.917',
 'parentCollection': None,
 'parentCollectionList': None,
 'parentCommunityList': None,
 'bitstreams': None,
 'archived': 'true',
 'withdrawn': 'false'}

In [96]:
udc_item = requests.get(rest_url + '/handle/11299/185533')
udc_item_json = json.loads(udc_item.content.decode('utf-8'))
udc_item_json

{'id': 63989,
 'name': '16S Intestinal Microbiome Sequences of Rhesus Macaques Treated with Chronic Morphine for 92 Days, SIV for 21 Days, or in Combination (Morphine for 70 Days then SIV+Morphine for 21 Days)',
 'handle': '11299/185533',
 'type': 'item',
 'link': '/rest/items/63989',
 'expand': ['metadata',
  'parentCollection',
  'parentCollectionList',
  'parentCommunityList',
  'bitstreams',
  'all'],
 'lastModified': '2017-04-19 12:55:53.382',
 'parentCollection': None,
 'parentCollectionList': None,
 'parentCommunityList': None,
 'bitstreams': None,
 'archived': 'true',
 'withdrawn': 'false'}

## URL examples

### Collections
/handle/{collectionId} - Return a collection with the specified ID.

* Bee Lab collection: https://conservancy.umn.edu/handle/11299/180876

* UMN Twin Cities collection: https://conservancy.umn.edu/handle/11299/1

* DRUM collection: https://conservancy.umn.edu/handle/11299/166578

### Authors?
* University of Minnesota Agricultuarl Experiment Station: https://conservancy.umn.edu/browse?type=author&value=University+of+Minnesota+Agricultural+Experiment+Station

* Dale C Dahl: https://conservancy.umn.edu/browse?type=author&value=Dahl,%20Dale%20C.

### Types?
* Conference Papers: https://conservancy.umn.edu/browse?type=type&value=Conference+Paper

### item records
GET /items/{item id} - Return the specified item.
* https://conservancy.umn.edu/handle/11299/136977

### search results?
* UDC search for school of journalism: https://conservancy.umn.edu/discover?scope=%2F&query=school+of+journalism&submit=
* DRUM search for agriculture: https://conservancy.umn.edu/handle/11299/166578/discover?query=agriculture&submit=
